In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
pd.set_option('display.max_columns', None)
college_park_coords = (38.9897, -76.9378)  # (lat, lon)
# Days of week
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [4]:
# Helper to convert "9am-5pm" -> "09:00", "17:00"
def parse_time(s):
    s = s.strip().lower()
    if 'am' in s or 'pm' in s:
        fmt = '%I:%M%p' if ':' in s else '%I%p'
    else:
        fmt = '%H:%M' if ':' in s else '%H'
    return datetime.strptime(s, fmt)


def parse_hours(hour_str):
    # Split on commas to separate multiple time ranges
    ranges = hour_str.split(',')
    times = []

    for time_range in ranges:
        open_raw, close_raw = time_range.strip().split('-')
        open_time = parse_time(open_raw)
        close_time = parse_time(close_raw)
        times.append((open_time, close_time))

    earliest_open = min(t[0] for t in times).strftime('%H:%M')
    latest_close = max(t[1] for t in times).strftime('%H:%M')

    return earliest_open, latest_close

# Function to compute simple spherical distance
def haversine(lat1, lon1, lat2, lon2):
    R = 3958.8  # Earth radius in miles
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

def opening_hours_df_to_dict(df):
    return {
        day.lower(): {'open': row['open'], 'close': row['close']}
        for day, row in df.iterrows()
    }


In [4]:
images_list = pd.read_excel('../datasets/FoodPantryImages.xlsx')['image_address']
images_list

0    https://streetviewpixels-pa.googleapis.com/v1/...
1    https://lh3.googleusercontent.com/p/AF1QipO2DB...
2    https://lh3.googleusercontent.com/p/AF1QipNpqT...
3    https://lh3.googleusercontent.com/p/AF1QipOgEU...
4    https://streetviewpixels-pa.googleapis.com/v1/...
5    https://streetviewpixels-pa.googleapis.com/v1/...
Name: image_address, dtype: object

In [11]:
df = pd.read_csv('./datasets/Food_Pantries_wGeo.csv')

df.head(20)

,Name,StreetAddress,City,State,ZipCode,Contact,Phone,StandardHours,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,ServiceTypes,Requirements,Languages,Accessibility,Latitude,Longitude,Notes
0,Abundant Life II,5110 Balto Natt Pike apt 4,Baltimore,MD,21229,Thomas Hill,(410) 947-6343,9am-12pm,1,0,0,0,0,0,0,NaN,NaN,NaN,NaN,39.2866,-76.7007,NaN
1,Adams Chapel AME,3813 Egerton Rd.,Baltimore,MD,21215,Arlean Burton,(410) 542-6200,11am-1pm,0,0,1,0,0,1,0,NaN,NaN,NaN,NaN,39.3337,-76.6897,NaN
2,Agape House,222 N. Carrollton Ave.,Baltimore,MD,21223,Rev. Robinson,(410) 728-2222,12pm-6pm,0,0,1,0,0,1,0,NaN,NaN,NaN,NaN,39.2912,-76.6447,NaN
3,AMES Memorial AME,615 Baker St.,Baltimore,MD,21211,Jacqueline Johnson,(410) 523-5556,11am-1pm,0,1,0,0,0,0,0,NaN,NaN,NaN,NaN,39.3086,-76.6334,NaN
4,Bernard E. Mason,2121 Windsor Gardens,Baltimore,MD,21207,Liflie Ziegler,(410) 448-1311,11am-4pm,0,0,0,1,0,0,0,NaN,NaN,NaN,NaN,39.3153,-76.6932,NaN
5,Bethany Baptist Church,2616 Ridgeley St.,Baltimore,MD,21230,Sanra White,(410) 539-5029,9am-12pm,1,1,1,1,1,0,0,NaN,NaN,NaN,NaN,39.2735,-76.6562,NaN
6,Bethel Church of God,301 W. 28th Street,Baltimore,MD,21211,Charletta Williams,(410) 235-0160,"9am-12pm, 12pm-3pm",0,0,0,0,1,0,1,NaN,NaN,NaN,NaN,39.3214,-76.6219,NaN
7,Carter Memorial Emergency,745 West Fayette St.,Baltimore,MD,21201,Verner Lewis,(410) 752-6123,10am-2pm,0,0,0,1,0,0,0,NaN,NaN,NaN,NaN,39.2904,-76.6278,NaN
8,Central Baptist Church,2035 West Baltimore St,Baltimore,MD,21223,Rev. Matthew Braxton,(410) 233-8558,12pm-2pm,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN,39.2870,-76.6447,NaN
9,Chase House,1027 Cathedral St,Baltimore,MD,21201,Joan Weston,(410) 539-6155,10am-2pm,0,0,0,0,1,0,0,NaN,NaN,NaN,NaN,39.3002,-76.6166,NaN


In [23]:
names = df['Name']
description = df['Notes'].fillna('')
address = df['StreetAddress']
latitude = df['Latitude']
longitude = df['Longitude']
contactPhone = df['Phone']
contactEmail = df['Contact']
imageUrl = np.random.choice(images_list, size=len(df))
walkingDistance = df.apply(
    lambda row: haversine(college_park_coords[0], college_park_coords[1], row['Latitude'], row['Longitude']),
    axis=1
)
offersDelivery = np.random.choice([True, False], size=len(df))
specialNotes = df['Notes'].fillna('')

In [24]:
# Result dictionary: {store name → day-by-day DataFrame}
openingHours = {}

for _, row in df.iterrows():
    name = row['Name']
    open_time, close_time = parse_hours(row['StandardHours'])

    # Filter only days where store is open
    open_days = [day for day in days if row[day] == 1]

    store_df = pd.DataFrame({
        'open': open_time,
        'close': close_time
    }, index=open_days)

    openingHours[name] = store_df


In [27]:
df1 = pd.DataFrame({
    'name': names.values,
    'description': description.values,
    'address': address.values,
    'latitude': latitude.values,
    'longitude': longitude.values,
    'contactPhone': contactPhone.values,
    'contactEmail': contactEmail.values,
    'imageUrl': imageUrl,
    'adminId': 1,
    'walkingDistance': walkingDistance.values,
    'offersDelivery': offersDelivery
})

df1['openingHours'] = df1['name'].map(openingHours)
df1['specialNotes'] = specialNotes
df1['openingHours'] = df1['openingHours'].apply(opening_hours_df_to_dict)

df1.head(12)

,name,description,address,latitude,longitude,contactPhone,contactEmail,imageUrl,adminId,walkingDistance,offersDelivery,openingHours,specialNotes
0,Abundant Life II,,5110 Balto Natt Pike apt 4,39.2866,-76.7007,(410) 947-6343,Thomas Hill,https://lh3.googleusercontent.com/p/AF1QipOgEU...,1,24.130458,False,"{'monday': {'open': '09:00', 'close': '12:00'}}",
1,Adams Chapel AME,,3813 Egerton Rd.,39.3337,-76.6897,(410) 542-6200,Arlean Burton,https://streetviewpixels-pa.googleapis.com/v1/...,1,27.232303,False,"{'wednesday': {'open': '11:00', 'close': '13:0...",
2,Agape House,,222 N. Carrollton Ave.,39.2912,-76.6447,(410) 728-2222,Rev. Robinson,https://lh3.googleusercontent.com/p/AF1QipO2DB...,1,26.089784,False,"{'wednesday': {'open': '12:00', 'close': '18:0...",
3,AMES Memorial AME,,615 Baker St.,39.3086,-76.6334,(410) 523-5556,Jacqueline Johnson,https://lh3.googleusercontent.com/p/AF1QipNpqT...,1,27.414150,False,"{'tuesday': {'open': '11:00', 'close': '13:00'}}",
4,Bernard E. Mason,,2121 Windsor Gardens,39.3153,-76.6932,(410) 448-1311,Liflie Ziegler,https://streetviewpixels-pa.googleapis.com/v1/...,1,26.036042,False,"{'thursday': {'open': '11:00', 'close': '16:00'}}",
5,Bethany Baptist Church,,2616 Ridgeley St.,39.2735,-76.6562,(410) 539-5029,Sanra White,https://streetviewpixels-pa.googleapis.com/v1/...,1,24.744629,True,"{'monday': {'open': '09:00', 'close': '12:00'}...",
6,Bethel Church of God,,301 W. 28th Street,39.3214,-76.6219,(410) 235-0160,Charletta Williams,https://lh3.googleusercontent.com/p/AF1QipOgEU...,1,28.490707,True,"{'friday': {'open': '09:00', 'close': '15:00'}...",
7,Carter Memorial Emergency,,745 West Fayette St.,39.2904,-76.6278,(410) 752-6123,Verner Lewis,https://lh3.googleusercontent.com/p/AF1QipNpqT...,1,26.601703,False,"{'thursday': {'open': '10:00', 'close': '14:00'}}",
8,Central Baptist Church,,2035 West Baltimore St,39.2870,-76.6447,(410) 233-8558,Rev. Matthew Braxton,https://lh3.googleusercontent.com/p/AF1QipOgEU...,1,25.858948,True,"{'saturday': {'open': '12:00', 'close': '14:00'}}",
9,Chase House,,1027 Cathedral St,39.3002,-76.6166,(410) 539-6155,Joan Weston,https://streetviewpixels-pa.googleapis.com/v1/...,1,27.504664,True,"{'friday': {'open': '10:00', 'close': '14:00'}}",


In [29]:
df1.to_json('output.json', orient='records', indent=2)

In [30]:
json_str = df1.to_json(orient='records', indent=2)
json_str = json_str.replace('\\/', '/')

# Save to file
with open('output.json', 'w') as f:
    f.write(json_str)